In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/ani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [3]:
data = pd.read_csv(r'../fake_or_real_news.csv')

/tmp/ipykernel_91913/2214948029.py:1: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,213,214,215,216,217,218,219,220,221,222,223,224,226,227,228,229,231,232,233,234,236,237,238,239,240,241,243,244,245,246,247,248,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,2

In [4]:
data = data.drop(['author'], axis = 1)

In [5]:
data = data.iloc[:, :3]
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
data = data.dropna()

In [7]:
data.loc[data.label != 'REAL', 'label'] = 1
data.loc[data.label == 'REAL', 'label'] = 0

In [8]:
y = data.iloc[:, -1:].values
y = [i for sub in y for i in sub]

In [9]:
def transform(c):
    return c["title"] + " " + c["text"]
x = data.apply(transform, axis=1)

In [10]:
def stemm(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [w for w in text if not w in stop_words]
    text = [stemmer.stem(word) for word in text]
    text = " ".join(text)
    return text

In [11]:
x = x.apply(stemm)
x

0        smell hillari fear daniel greenfield shillman ...
1        watch exact moment paul ryan commit polit suic...
2        kerri go pari gestur sympathi u secretari stat...
3        berni support twitter erupt anger dnc tri warn...
4        battl new york primari matter primari day new ...
                               ...                        
70795    tesla earn smash expect dramat chang report me...
70796    rule ruler world realli work follow video must...
70797    fact check trump right clinton might caus ww e...
70798    caught tape isi destroy iraqi abram anti tank ...
70799    obamacar architect admit law work design premi...
Length: 64733, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

In [13]:
c_vec = CountVectorizer(max_features=5000)
tfidf_vec = TfidfVectorizer(max_features=5000)
lr_model = LogisticRegression()

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)

In [15]:
x_train = c_vec.fit_transform(x_train).toarray()
x_test = c_vec.transform(x_test).toarray()

In [16]:
def logReg(x, y):
  lr_model.fit(x_train, y_train)
  y_trainPred = lr_model.predict(x_train)
  y_testPred = lr_model.predict(x_test)
  print("Accuracy of training = ", accuracy_score(y_trainPred, y_train))
  print('Training accuracy\n')
  print(classification_report(y_train, y_trainPred))
  print("Accuracy of Testing = ", accuracy_score(y_testPred, y_test))
  print('Testing Results\n')
  print(classification_report(y_test, y_testPred))

In [17]:
logReg(x_train, y_train)

/home/ani/Aiproj/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of training =  0.9976248406905341
Training accuracy

              precision    recall  f1-score   support

           0       1.00      0.95      0.97      2519
           1       1.00      1.00      1.00     49267

    accuracy                           1.00     51786
   macro avg       1.00      0.98      0.99     51786
weighted avg       1.00      1.00      1.00     51786

Accuracy of Testing =  0.9752066115702479
Testing Results

              precision    recall  f1-score   support

           0       0.80      0.67      0.73       642
           1       0.98      0.99      0.99     12305

    accuracy                           0.98     12947
   macro avg       0.89      0.83      0.86     12947
weighted avg       0.97      0.98      0.97     12947

